In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model


In [2]:
insurance = pd.read_csv("/kaggle/input/insurance/insurance.csv")
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

ct = make_column_transformer(
    (MinMaxScaler(),["age","bmi","children"]),
    (OneHotEncoder(handle_unknown="ignore"),["sex","smoker","region"])
)

X = insurance.drop("charges", axis=1)
y = insurance["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

len(X), len(X_train), len(X_test)

(1338, 1070, 268)

In [4]:
ct.fit(X_train)

X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [5]:
X_train.shape, X_train_normal.shape, y_train.shape

((1070, 6), (1070, 11), (1070,))

In [6]:
tf.random.set_seed(42)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

insurance_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1)
])

insurance_model.compile(loss = tf.keras.losses.mae,
                       optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
                       metrics = ["mae"])

insurance_model.fit(X_train_normal, y_train, epochs = 200,callbacks=[callback], verbose = 1)
#insurance_model.summary()

Epoch 1/200
34/34 [==============================] - 1s 2ms/step - loss: 12226.4707 - mae: 12226.4707
Epoch 2/200
34/34 [==============================] - 0s 2ms/step - loss: 7531.9077 - mae: 7531.9077
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 5193.0327 - mae: 5193.0327
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 3595.0076 - mae: 3595.0076
Epoch 5/200
34/34 [==============================] - 0s 2ms/step - loss: 3363.8933 - mae: 3363.8933
Epoch 6/200
34/34 [==============================] - 0s 2ms/step - loss: 3295.9070 - mae: 3295.9070
Epoch 7/200
34/34 [==============================] - 0s 2ms/step - loss: 3249.8777 - mae: 3249.8777
Epoch 8/200
34/34 [==============================] - 0s 2ms/step - loss: 3252.8889 - mae: 3252.8889
Epoch 9/200
34/34 [==============================] - 0s 2ms/step - loss: 3160.5444 - mae: 3160.5444
Epoch 10/200
34/34 [==============================] - 0s 2ms/step - loss: 3133.3240 - mae: 3133.32

In [7]:
insurance_model.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 1777.8735 - mae: 1777.8735


[1777.87353515625, 1777.87353515625]